In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import cv2
import os
import shutil

from glob import glob
from pathlib import Path

In [4]:
!pip install wolta

  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp39-cp39-win_amd64.whl.metadata (168 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.8 MB 540.5 kB/s eta 0:03:08
   ---------------------------------------- 0.5/101.8 MB 540.5 kB/s eta 0:03:08
   ---------------------------------------- 0.8/101.8 MB 609.6 kB/s eta 0:02:46
   ---------------------------------------- 0.8/101.8

In [5]:
!pip install ultralytics

  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/898.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/898.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/898.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/898.7 kB ? eta -:--:--
   ---------------------- --------------- 524.3/898.7 kB 419.4 kB/s eta 0:00:01
   ---------------------- --------------- 524.3/898.7 kB 419.4 kB/s eta 0:00:01
   -------------------------------------- 898.7/898.7 kB 608.0 kB/s eta 0:00:00
Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.5.2 requires google-ai-generativelanguage==0.6.2, which is not installed.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [6]:
os.environ['WANDB_MODE'] = 'disabled'

# Data Analysis

In [ ]:
for dirname, _, _ in os.walk('kaggle/input'):
    print(dirname)

In [11]:
p_paths = glob('kaggle/input/*')
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))
print(d_paths)

['kaggle/input\\test\\FAKE', 'kaggle/input\\test\\REAL', 'kaggle/input\\train\\FAKE', 'kaggle/input\\train\\REAL']


In [12]:
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

120000


In [13]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{'jpg': 120000}

In [14]:
from wolta.visual_tools import dataset_size_same

dataset_size_same(i_paths)

True

In [15]:
temp_img = cv2.imread(i_paths[0])
ratio = temp_img.shape[1] / temp_img.shape[0]

print('Width: {}'.format(temp_img.shape[1]))
print('Height: {}'.format(temp_img.shape[0]))
print('Ratio: {}'.format(ratio))

Width: 32
Height: 32
Ratio: 1.0


# Image Stacking

In [16]:
os.makedirs('kaggle/working/raw')

In [18]:
for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = 'kaggle/working/raw/{}'.format(current_dir) 
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))

    for i_path in i_paths:
        shutil.copy(i_path, current_path)

# Image Splitting

In [19]:
from wolta.visual_tools import dir_split

dir_split('kaggle/working/raw', 'kaggle/working/data', test_size=0.2, val_size=0.2)

In [21]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('kaggle/working/data')

{'FAKE': 60000, 'REAL': 50000}

In [3]:

from tqdm import tqdm
# import standard scaling
#  !pip install scikit-learn
from sklearn.preprocessing import StandardScaler
n_dropped_patches = 0
def add_gaussian_noise(image, mean=0, std=25):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image


def reduce_resolution(image, target_size=(16, 16)):
    """Reduce and then restore the resolution of an image."""
    low_res = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    restored = cv2.resize(low_res, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
    return restored

def extract_patches(image, patch_size=1):
    """
    Extract patches from an image.
    Returns a list of flattened patch intensities.
    """
    height, width, _ = image.shape
    patches = []
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size].flatten()
            patches.append(np.mean(patch))  # Use mean intensity as a feature
    return patches

def compute_global_correlation(data_dir, categories, patch_size=1):
    """
    Compute the correlation of patches across the entire dataset with the label.
    """
    all_features = []
    all_labels = []

    for category, label in categories.items():
        category_path = os.path.join(data_dir, category)
        for img_name in tqdm(os.listdir(category_path)):
            img_path = os.path.join(category_path, img_name)
            if os.path.isfile(img_path):
                image = cv2.imread(img_path)
                if image is not None:
                    features = extract_patches(image, patch_size)
                    all_features.append(features)
                    all_labels.append(label)
    
    # standard scaling
    all_features = StandardScaler().fit_transform(all_features)
    # Convert to a DataFrame for correlation calculation
    feature_matrix = pd.DataFrame(all_features)  # Rows = images, Columns = patches
    print("Feature_Matrix.Head()..........")
    print(feature_matrix.head())
    label_series = pd.Series(all_labels, name="label")
    # print(label_series)
    correlations = feature_matrix.corrwith(label_series)  # Correlation of each patch with the label
    print("Correlation.Head()...........")
    print(correlations.head())
    print(f"Max correlation: {abs(correlations.max())}")
    print(f"Min correlation: {abs(correlations.min())}")
    print(f"Max correlation index: {abs(correlations).idxmax()}")
    print(f"Size of correlation: {correlations.size}")

    return correlations

def drop_least_correlated_features(image, correlations, patch_size=1, threshold=0.1):
    """
    Drop patches with the least correlation to the label.
    """
    height, width, _ = image.shape
    dropped_image = image.copy()
    patch_idx = 0

    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            if patch_idx < len(correlations) and abs(correlations[patch_idx]) < threshold:
                dropped_image[i:i+patch_size, j:j+patch_size] = 0  # Drop patch (set to black)
                # n_dropped_patches += 1
            patch_idx += 1

    return dropped_image

def process_dataset_with_correlation(data_dir, output_dir, correlations, patch_size=1, threshold=0.1):
    """
    Process the entire dataset by dropping least correlated patches and saving.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


    for img_name in tqdm(os.listdir(data_dir)):
        img_path = os.path.join(data_dir, img_name)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is not None:
                processed_image = drop_least_correlated_features(image, correlations, patch_size, threshold)
                output_img_path = os.path.join(output_dir, img_name)
                cv2.imwrite(output_img_path, processed_image)
                    


def normalize_image(image):
    """Normalize image to range [0, 1]."""
    return image / 255.0
    
def process_and_save_images(input_path, output_path, preprocess_fn):
    """Process images with a given preprocessing function and save."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    image_paths = os.listdir(input_path)
    for img_name in tqdm(image_paths):
        img_path = os.path.join(input_path, img_name)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is None:
                continue
            processed_image = preprocess_fn(image)
            img_name = img_name.split('.')[0] + '_1.' + img_name.split('.')[1]
            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, (processed_image * 255).astype(np.uint8) if processed_image.max() <= 1 else processed_image)


base_dir = "kaggle/working/data"  # Replace with the root path of your dataset
subsets = [ "train","val"]
all_subsets = [ "train","val", "test"]

categories = {"REAL": 1, "FAKE": 0}
train_data_dir = "kaggle/working/data/train"  # Training data directory

test_real_path = "kaggle/working/data/test/REAL"
test_fake_path = "kaggle/working/data/test/FAKE"





# print(f"Processing test/REAL...")
# process_and_save_images(test_real_path, test_real_path, lambda img: add_gaussian_noise(img))
# print(f"Processing test/FAKE...")
# process_and_save_images(test_fake_path, test_fake_path, lambda img: add_gaussian_noise(img))




print("Computing global correlation...")
correlations = compute_global_correlation(train_data_dir, categories, patch_size=1)


# # drop least correlated features from all sets 
# for subset in all_subsets:
#     for category, label in categories.items():
#         # n_dropped_patches = 0
#         input_path = os.path.join(base_dir, subset, category)        
#         print(f"Processing {subset}/{category} for feature dropping...")
#         process_dataset_with_correlation(input_path, input_path, correlations, patch_size=1, threshold=0.015)
#         # print(f"Dropped {n_dropped_patches} patches in total in images of {subset}/{category}.")

# # # Apply the Robustness Methods to train and val set
# # for subset in subsets:
# #     for category, label in categories.items():
# #         input_path = os.path.join(base_dir, subset, category)
#         # output_path = os.path.join(base_dir, f"{subset}_{category}_processed")
        
#         # print(f"Processing {subset}/{category} for reduced resolution...")
#         # process_and_save_images(input_path, input_path, lambda img: reduce_resolution(img))
        
        
#         # print(f"Processing {subset}/{category} for normalization...")
#         # process_and_save_images(input_path, input_path, lambda img: normalize_image(img))



Computing global correlation...


100%|██████████| 36000/36000 [12:37<00:00, 47.55it/s]


Feature_Matrix.Head()..........
       0         1         2         3         4         5         6     \
0  0.844898  0.400667 -0.269954 -0.227860  0.191736  0.615393  1.076511   
1  0.954245  1.055050  1.118452  1.152808  1.207978  1.252704  1.199755   
2  0.188820 -0.090120 -0.318330 -0.363982 -0.169811  0.345761  0.756078   
3 -1.227930 -1.278594 -1.319724 -1.282807 -1.195825 -1.198493 -1.334132   
4 -0.709723 -0.980273 -1.034303 -0.845271 -0.643732 -0.384696 -0.180573   

       7         8         9     ...      1014      1015      1016      1017  \
0  1.088716  0.441445  0.063904  ... -1.344942 -1.191971 -0.618905 -0.388929   
1  1.098588  1.158567  1.213386  ... -0.287648 -0.436500 -0.512695 -0.501102   
2  0.792559  0.367260 -0.396880  ... -0.252208 -0.224023 -0.152761 -0.158679   
3 -1.492788 -1.413181 -1.392767  ... -0.671582 -0.890963 -0.736917 -0.896661   
4 -0.194632  0.075465 -0.045099  ... -0.393968 -0.330261 -0.247170 -0.465679   

       1018      1019      1020     

In [13]:
print(f"Correlations under 0.015: {len(correlations[correlations < 0.02])}")

Correlations under 0.015: 119


In [24]:
# shutil.rmtree('kaggle/working/data')
# shutil.copytree('kaggle/working/data_original', 'kaggle/working/data')

KeyboardInterrupt: 

In [ ]:
# Drop 80% of the images in the training set
for subset in all_subsets:
    for category, label in categories.items():
        input_path = os.path.join(base_dir, subset, category)
        image_paths = os.listdir(input_path)
        if len(image_paths) > 0.2 * len(image_paths):
            image_paths = image_paths[:int(0.8 * len(image_paths))]
            for img_name in image_paths:
                img_path = os.path.join(input_path, img_name)
                os.remove(img_path)

In [15]:
# n_dropped_patches = 0
# def drop_least_correlated_features(image, correlations, patch_size=1, threshold=0.1):
#     """
#     Drop patches with the least correlation to the label.
#     """
#     height, width, _ = image.shape
#     dropped_image = image.copy()
#     patch_idx = 0
#     n_dropped_patches = 0
#     for i in range(0, height, patch_size):
#         for j in range(0, width, patch_size):
#             if patch_idx < len(correlations) and abs(correlations[patch_idx]) < threshold:
#                 dropped_image[i:i+patch_size, j:j+patch_size] = 0  # Drop patch (set to black)
#                 n_dropped_patches += 1
#             patch_idx += 1
#     print(f"Dropped {n_dropped_patches} patches in this image")
#     return dropped_image, n_dropped_patches
# def process_dataset_with_correlation(data_dir, output_dir, correlations, patch_size=1, threshold=0.1):
#     """
#     Process the entire dataset by dropping least correlated patches and saving.
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     n_dropped_patches = 0
#     for img_name in tqdm(os.listdir(data_dir)):
#         img_path = os.path.join(data_dir, img_name)
#         if os.path.isfile(img_path):
#             image = cv2.imread(img_path)
#             if image is not None:
#                 processed_image, n_d = drop_least_correlated_features(image, correlations, patch_size, threshold)
#                 n_dropped_patches += n_d
#                 output_img_path = os.path.join(output_dir, img_name)
#                 cv2.imwrite(output_img_path, processed_image)
#     return n_dropped_patches 
                
# all_subsets = [ "train","val", "test"]

# # drop least correlated features from all sets 
# for subset in all_subsets:
#     for category, label in categories.items():
#         n_dropped_patches = 0
#         input_path = os.path.join(base_dir, subset, category)   
#         output_path = os.path.join(base_dir, subset, category+"_processed")
#         print(f"Processing {subset}/{category} for feature dropping...")
#         n_dropped_patches = process_dataset_with_correlation(input_path, output_path, correlations, patch_size=1, threshold=0.035)
#         print(f"Dropped {n_dropped_patches} patches in total in images of {subset}/{category}.")

In [16]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('kaggle/working/data')

{'FAKE': 168000, 'REAL': 140000}

# YOLO

In [17]:
from ultralytics import YOLO

model = YOLO(model='yolo11x-cls.pt')
results = model.train(data='kaggle/working/data', epochs=5, imgsz=32, verbose= True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 56.9M/56.9M [00:00<00:00, 115MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.pt, data=/kaggle/working/data, epochs=5, time=None, patience=100, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

2024-12-16 01:59:20,770	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-16 01:59:21,252	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

...

               classes   top1_acc   top5_acc: 100%|██████████| 2063/2063 [00:39<00:00, 51.64it/s]

                   all      0.896          1



5 epochs completed in 1.122 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 57.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 57.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 198000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 66000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 2063/2063 [00:30<00:00, 68.15it/s]


                   all      0.896          1
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [18]:
test_results = model.val(data='kaggle/working/data', imgsz=32, split="test")

Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 198000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 66000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


test: Scanning /kaggle/working/data/test... 44000 images, 0 corrupt: 100%|██████████| 44000/44000 [00:20<00:00, 2185.55it/s]


test: New cache created: /kaggle/working/data/test.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 2750/2750 [00:31<00:00, 88.55it/s]


                   all      0.678          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [19]:
# Save the trained model
model.save('trained_yolo_model.pt')
shutil.rmtree('kaggle/working/data')
shutil.rmtree('kaggle/working/raw')
